In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/toxic_comments.csv")

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df= df.head(5000)

In [5]:
df.toxic.value_counts()

0    4462
1     538
Name: toxic, dtype: int64

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score


In [7]:

from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [43]:
model = model.to('cuda')

In [10]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [11]:
df.dropna(inplace=True)

In [12]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [13]:
X = list(df['text'])
y= list(df['toxic'])

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y)
X_trained_tokanized = tokenizer(X_train, padding=True , truncation= True, max_length= 512)
X_test_tokanized = tokenizer(X_test, padding=True , truncation= True, max_length= 512)

In [15]:
X_trained_tokanized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels= None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [17]:
train_dataset = Dataset(X_trained_tokanized ,y_train)
test_dataset = Dataset(X_test_tokanized, y_test)

In [18]:
train_dataset[5]

{'input_ids': tensor([  101,  2054,  2515,  2023, 15536,  3211, 21572, 20614,  2086,  2812,
          1029,  2017,  4778,  2033,  1012,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [19]:
def compute_metrics(p):
  pred, labels = p
  pred =np.argmax(pred, axis =1)

  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels, y_pred=pred)

  return {
      'accuracy' : accuracy,
      'precision' : precision,
      'recall' : recall,
      'f1' : f1
  }

In [20]:
arg = TrainingArguments(
    output_dir = 'output',
    num_train_epochs = 10,
    per_device_train_batch_size = 8

)

trainer = Trainer(
    model = model,
    args = arg,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

Step,Training Loss
500,0.093800
1000,0.047200
1500,0.025300
2000,0.005700
2500,0.000500
3000,0.000000


KeyboardInterrupt: 

In [23]:
trainer.evaluate()

{'eval_loss': 0.4234803318977356,
 'eval_accuracy': 0.95,
 'eval_precision': 0.7636363636363637,
 'eval_recall': 0.7777777777777778,
 'eval_f1': 0.7706422018348624}

In [32]:
text = "This product is a very very bad product"
inputs = tokenizer(text,padding=True,truncation=True, return_tensors ="pt").to('cuda')

In [33]:
output= model(**inputs)

In [34]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.4601, -6.0460]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [35]:
prediction = torch.nn.functional.softmax(output.logits,dim=-1)

In [36]:
prediction

tensor([[9.9999e-01, 1.0069e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [37]:
predictions =prediction.cpu().detach().numpy()

In [41]:
predictions

array([[9.9998999e-01, 1.0068909e-05]], dtype=float32)

In [38]:
pred= np.argmax(predictions)

In [39]:
pred

0

In [40]:
trainer.save_model("CustomModel")